In [1]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText, Word2Vec
from tqdm.notebook import tqdm

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
sales = pd.read_csv("./data.csv", low_memory=False)

In [3]:
sales

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"
...,...,...,...,...,...,...,...,...,...
19999998,2018-06-13,1924036,1485.0,71723.0,(60907) Тералиджен табл. п.п.о. 5 мг №25,637.0,146.0,-1.0,"1,00"
19999999,2018-06-13,1788779,1503.0,66164.0,(27922) Дифлюкан капс. 150 мг. №1,622.0,142.0,-1.0,"1,00"
20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20000001,(затронуто стр,к: 20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
sales[sales['sale_date_date'].isna()]

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sales.dropna(inplace=True)
sales.reset_index(inplace=True, drop=True)
sales.sort_values('sale_date_date', inplace=True)

In [6]:
# Создадим колонку, оккумулирующию данные по чеку: 
# время + касса + магазин
sales['receipt_sum'] = (sales['sale_date_date'].apply(str) + "_" + 
                        sales['contact_id'].apply(str) + "_" + 
                        sales['shop_id'].apply(str))

In [7]:
receipts = {}

for i in range(len(sales)):
    if sales['receipt_sum'].iloc[i] not in receipts.keys():
        receipts[sales['receipt_sum'].iloc[i]] = []
        
    receipts[sales['receipt_sum'].iloc[i]].append(str(sales['product_id'].iloc[i]))

In [8]:
receipts

{'2018-01-01_127175_603.0': ['153822.0', '91259.0'],
 '2018-01-01_1786329_1482.0': ['168570.0',
  '92768.0',
  '216024.0',
  '81067.0',
  '105144.0',
  '79455.0'],
 '2018-01-01_1326823_1522.0': ['112760.0',
  '130823.0',
  '51861.0',
  '139270.0',
  '135993.0',
  '40259.0',
  '118050.0'],
 '2018-01-01_1556808_1530.0': ['164413.0', '70811.0', '70403.0', '113932.0'],
 '2018-01-01_1383494_1516.0': ['123127.0',
  '221520.0',
  '168308.0',
  '90747.0',
  '146081.0',
  '23228.0',
  '28118.0'],
 '2018-01-01_1685583_1473.0': ['161378.0', '168308.0', '147953.0', '144155.0'],
 '2018-01-01_1786605_1462.0': ['126016.0', '168570.0', '130595.0'],
 '2018-01-01_1294874_1511.0': ['168570.0',
  '120130.0',
  '107192.0',
  '129591.0',
  '72789.0',
  '119526.0'],
 '2018-01-01_1439514_1528.0': ['72448.0', '106107.0', '33206.0'],
 '2018-01-01_1786829_1490.0': ['112405.0',
  '112405.0',
  '50667.0',
  '134990.0',
  '112405.0',
  '134990.0',
  '168320.0'],
 '2018-01-01_1335622_1457.0': ['49944.0',
  '168570.0

In [10]:
len(receipts)

6123562

In [11]:
sales['all_items'] = sales['receipt_sum'].apply(lambda x: receipts[x])

In [12]:
receipts_df = pd.Series(receipts)

In [13]:
receipts_df = pd.Series.to_frame(receipts_df)

In [14]:
receipts_df

,0
2018-01-01_127175_603.0,"[153822.0, 91259.0]"
2018-01-01_1786329_1482.0,"[168570.0, 92768.0, 216024.0, 81067.0, 105144...."
2018-01-01_1326823_1522.0,"[112760.0, 130823.0, 51861.0, 139270.0, 135993..."
2018-01-01_1556808_1530.0,"[164413.0, 70811.0, 70403.0, 113932.0]"
2018-01-01_1383494_1516.0,"[123127.0, 221520.0, 168308.0, 90747.0, 146081..."
...,...
2018-12-09_1303877_1468.0,"[65663.0, 90241.0]"
2018-12-09_2567928_1511.0,"[109489.0, 120163.0, 116291.0]"
2018-12-09_2746260_132.0,"[75997.0, 154091.0, 23678.0, 36001.0, 166672.0..."
2018-12-09_2224486_134.0,"[144820.0, 162897.0, 138005.0]"


In [15]:
import pickle


with open('Product_dict.pkl', 'rb') as f:
    prod_dict = pickle.load(f)

prod_dict

{'168308': '(197312) Пакет-майка 25см х 45см 906',
 '134832': '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404',
 '101384': '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404',
 '168570': '(197309) Пакет 28см х 50см 906',
 '146960': '(111023) Пакет "Аптека Озерки" 28 х 35см 906',
 '128700': '(51806) Ранитидин тб 150мг уп N20 595',
 '52539': '(19416) Платки носовые "Зева Кидс" №10 721',
 '94270': '(66684) Пакет майка "Благодарим за покупку" (28+14) х 43см №1 -1',
 '77426': '(5605) Иглы Микро-Файн д/шприц-ручки 30G 0,3*8мм N100 759',
 '92475': '(114214) Дорзопт капли глазные 2% 5мл. фл.-кап. N3 575',
 '158643': '(181230) PL Шприц одноразовый 3-комп. 3мл №1 762',
 '158600': '(181232) PL Шприц одноразовый 3-комп. 5мл №1 762',
 '106230': '(43344) Аллапинин тб 25мг N30 738',
 '110629': '(57733) Корвалол капли д/приема внутрь 25мл 738',
 '47251': '(121286) PL Контейнер д/биопроб стер. 60мл с крыш и ложкой в инд.уп 711',
 '39811': '(116543) PL Салфетка антисептическая спи

Разделение датасета на тренировочный и валидационный

In [16]:
# Разделяем данные на тренироваочный и валидационный датасеты таким образом, что в валидационном датасете данные 
# о более поздних покупках
train, test = sales[:14028861], sales[14028861:]  # train_size = 0.3

In [17]:
sales['sale_date_date'].iloc[14028860], sales['sale_date_date'].iloc[14028861]

('2018-06-09', '2018-06-10')

In [18]:
sales[sales['sale_date_date'] > "2018-06-09"]

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity,receipt_sum,all_items
19651963,2018-06-10,864888,538.0,25220.0,(113523) Фурацилин Авексима таб. шип. д/р-ра 2...,403.0,92.0,-1.0,"1,00",2018-06-10_864888_538.0,"[25220.0, 161495.0, 51319.0]"
19651962,2018-06-10,171107,999.0,98911.0,(90927) Доктор Столетов Вата мед хирург н/стер...,672.0,154.0,3308.0,"1,00",2018-06-10_171107_999.0,"[98911.0, 115534.0, 76063.0, 59701.0, 116562.0..."
19651908,2018-06-10,132252,346.0,143859.0,(103962) Назонекс наз спрей 50мкг/доза 120доз N1,701.0,162.0,-1.0,"1,00",2018-06-10_132252_346.0,"[143859.0, 119217.0, 89638.0, 161451.0, 123869..."
19651907,2018-06-10,446343,600.0,98270.0,(14991) Реню МПС р-р д/ухода за контактными ли...,817.0,88.0,479.0,"1,00",2018-06-10_446343_600.0,"[98270.0, 39186.0, 88564.0, 109932.0, 55522.0,..."
19651906,2018-06-10,458743,348.0,89581.0,(115797) Гроссхертц Витамин С 900мг таб.шип.№20,429.0,101.0,2038.0,"1,00",2018-06-10_458743_348.0,"[89581.0, 96651.0, 85078.0]"
...,...,...,...,...,...,...,...,...,...,...,...
115256,2018-12-09,2419062,1739.0,151054.0,(8837) Джонсонс Бэби масло Ромашка 200мл,774.0,177.0,3239.0,"1,00",2018-12-09_2419062_1739.0,[151054.0]
115255,2018-12-09,2742143,66.0,105192.0,(120201) Пакет бумажный 120х240х280мм с печать...,906.0,205.0,-1.0,"1,00",2018-12-09_2742143_66.0,"[75509.0, 105192.0, 99004.0]"
115254,2018-12-09,2742143,66.0,99004.0,(35198) Бифилиз пор 5дз N10,595.0,135.0,-1.0,"1,00",2018-12-09_2742143_66.0,"[75509.0, 105192.0, 99004.0]"
115266,2018-12-09,2224486,134.0,138005.0,(26822) Лизобакт тб д/рассасыв №30,404.0,93.0,-1.0,"1,00",2018-12-09_2224486_134.0,"[144820.0, 162897.0, 138005.0]"


# Kонтетные рекомендации

In [19]:
# Предобработка текста

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

# The function for text processing
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [20]:
sentences = [preprocess_txt(k) for k in prod_dict.keys()]

In [21]:
# Подготовим кандидатогенератор, который будет выбирать товары
# на основании товаров в чеке

modelFT = FastText(sentences=sentences, vector_size=20, 
                   min_count=1, window=5)


# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

In [22]:
modelFT.wv['пакет']

array([ 5.4020439e-03, -1.3947221e-06,  1.7954146e-02,  5.9474871e-04,
        9.4182044e-03, -8.0221640e-03,  6.7024906e-03, -1.0572030e-02,
       -1.3165076e-02,  3.5183642e-03, -2.0834139e-02, -1.7274285e-03,
       -6.3115865e-04, -1.2223838e-03, -1.0253174e-02,  1.2972181e-03,
       -9.7466838e-03, -2.5485037e-04,  2.4473949e-03, -6.6153225e-03],
      dtype=float32)

In [23]:
reverse_index_map = {}
counter = 0

for name in tqdm(prod_dict.keys()):
    n_ft = 0
    reverse_index_map[counter] = name
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt(name):
        if word in modelFT.wv.key_to_index.keys():
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1

ft_index_names.build(10)

  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [24]:
reverse_index_map

{0: '168308',
 1: '134832',
 2: '101384',
 3: '168570',
 4: '146960',
 5: '128700',
 6: '52539',
 7: '94270',
 8: '77426',
 9: '92475',
 10: '158643',
 11: '158600',
 12: '106230',
 13: '110629',
 14: '47251',
 15: '39811',
 16: '143709',
 17: '25965',
 18: '158645',
 19: '79879',
 20: '100667',
 21: '87332',
 22: '21249',
 23: '158659',
 24: '76206',
 25: '114366',
 26: '34531',
 27: '27063',
 28: '20236',
 29: '99525',
 30: '116091',
 31: '97117',
 32: '160521',
 33: '94899',
 34: '67922',
 35: '98991',
 36: '90247',
 37: '28232',
 38: '60404',
 39: '19578',
 40: '124839',
 41: '42409',
 42: '77808',
 43: '60228',
 44: '110342',
 45: '124535',
 46: '79735',
 47: '68982',
 48: '135018',
 49: '141455',
 50: '148629',
 51: '55357',
 52: '135427',
 53: '19510',
 54: '66818',
 55: '141996',
 56: '161562',
 57: '155688',
 58: '155686',
 59: '162991',
 60: '87663',
 61: '145608',
 62: '108808',
 63: '127911',
 64: '141757',
 65: '153569',
 66: '115144',
 67: '32087',
 68: '71618',
 69: '114

In [27]:
def recommend(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for item in items_list:
        if item not in prod_dict.keys():
            break
        item_name = prod_dict[item]
        if item_name not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector
                                   [reverse_index_map[item_name]])
    return [reverse_index_map[i] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [28]:
recomended = recommend(['52539', '110342'])

for num in recomended:
    print(f'{num}: {prod_dict[num]}')

125133: (106357) Салфетки EURON д/ухода за лежачими больными №64 755
96067: (79205) Верапамил тб п/о 80мг N30 738
76638: (5061) Лейкопластырь 2см х 500см катушка в уп.из полист. N1 670
77933: (64790) Ксимелин ЭКО с ментолом спрей назальный 140мкг/доза 1мг/мл фл.10мл 701
165059: (194632) Корега крем для фиксации зубных протезов Максимальное прилегание 40г 533
86035: (63827) Аркоксиа табл. п.п.о.90мг №7 647
89839: (91731) Селенцин шампунь от выпадения волос 200мл 476
139039: (112926) Либридерм Пантенол Форте 9% крем 30мл 547
59775: (91616) Гевискон Двойное Действие сусп.д/приема внутрь (мятная) пакет 10мл №12 595
165119: (196468) СалиЦинк Пенка для умывания для жирной и комбинированной кожи Сера-Цинк 160мл 503


# Рекомендации по чековым данным

In [36]:
# Обучение эмбеддингов по последовательности
train['product_id'] = train['product_id'].apply(int).apply(str)

In [37]:
# Сформируем последовательности

grouped = train.groupby('receipt_sum')
sentences = []

for group in tqdm(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentences.append(list(products))

  0%|          | 0/4119309 [00:00<?, ?it/s]

In [38]:
sentences

[['62087', '120171', '69028'],
 ['37070', '115617', '142296'],
 ['138583', '52529', '45321', '134475', '99821', '82717'],
 ['144740', '113241', '70976', '44789'],
 ['21326', '79960', '112688'],
 ['137539', '70772', '134394', '37779', '108353'],
 ['68418', '20236', '129521', '51228'],
 ['44453', '40672', '66296'],
 ['51013', '113769', '86275'],
 ['19543', '113585', '56622', '92304'],
 ['75329', '51861', '141161', '69616'],
 ['131863', '21252', '142938', '77651', '148319', '94105'],
 ['137539',
  '118593',
  '78852',
  '105944',
  '105944',
  '74433',
  '100907',
  '127362',
  '41890'],
 ['98150', '83870', '74585'],
 ['102814', '85456', '141059', '144743'],
 ['146882', '93853', '55888', '70496', '82793', '98132', '144051'],
 ['143655', '135114', '156491'],
 ['113585', '102991', '138232'],
 ['46797', '129521', '81249', '120841'],
 ['77808', '106107', '128672'],
 ['148104', '102048', '143445'],
 ['65984', '131301', '51951'],
 ['71384', '79507', '78746', '70642'],
 ['52106', '117372', '7996

In [39]:
modelW2V = Word2Vec(sentences, vector_size=20)

In [40]:
def recommend_w2v(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for item in items_list:
        if item not in prod_dict.keys():
            break
        item_name = prod_dict[item]
        if item_name not in modelW2V.wv.key_to_index.keys():
            c -= 1
            continue
        current_vector += np.array(modelW2V.wv[reverse_index_map[item_name]])
    return [reverse_index_map[i] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [41]:
recomended = recommend_w2v(['52539', '110342'])

for num in recomended:
    print(f'{num}: {prod_dict[num]}')

125133: (106357) Салфетки EURON д/ухода за лежачими больными №64 755
96067: (79205) Верапамил тб п/о 80мг N30 738
76638: (5061) Лейкопластырь 2см х 500см катушка в уп.из полист. N1 670
77933: (64790) Ксимелин ЭКО с ментолом спрей назальный 140мкг/доза 1мг/мл фл.10мл 701
165059: (194632) Корега крем для фиксации зубных протезов Максимальное прилегание 40г 533
86035: (63827) Аркоксиа табл. п.п.о.90мг №7 647
89839: (91731) Селенцин шампунь от выпадения волос 200мл 476
139039: (112926) Либридерм Пантенол Форте 9% крем 30мл 547
59775: (91616) Гевискон Двойное Действие сусп.д/приема внутрь (мятная) пакет 10мл №12 595
165119: (196468) СалиЦинк Пенка для умывания для жирной и комбинированной кожи Сера-Цинк 160мл 503
